Title

## Introduction

TODO : explain what this is for

Explain that you can skip if you are already familiar with xxx

## Setup

In [2]:
# Specify path for imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## 1. Dataset overview

> Let's first have a quick overview on what the dataset we will work on should look like, and what is necessary for the package functions to be workable.

We will work with **two types of datasets**, depending on whether you need statistics at the **issues level** or at the **content item level**.

### 1.1  Issues table (SQL > pandas)
For the analysis at the issues level, data has been loaded from SQL. Here are some tool functions that can be useful if you also wish to load data from SQL.

##### Read Table
The `read_table` function is usefull for loading a table from SQL to pandas. You can use it if you have the database in SQL format. Otherwise you should load your database the way you are used to, to have it as a pandas dataframe.

The `db_engine` function is needed for creating the sql engine from you environment variables. Check in the code directly to see how your environment varibles should be named.

In [6]:
from impresso_stats.sql import db_engine, read_table

In [7]:
engine = db_engine()

As most function are made to do analysis at the issue level (or content item level but we will see that later), we focus on the `issues` table, but you can use this function to load any other impression SQL table (e.g. `newspapers` table).

In [28]:
issues_df = read_table('impresso.issues', engine)
#newspapers_df = read_table('impresso.newspapers', engine)

##### Snapshot

In [10]:
issues_df.head()

,id,year,month,day,edition,access_rights,created,last_modified,is_damaged,s3_version,newspaper_id
0,actionfem-1927-10-15-a,1927,10,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem
1,actionfem-1927-11-15-a,1927,11,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem
2,actionfem-1927-12-15-a,1927,12,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem
3,actionfem-1928-01-15-a,1928,1,15,a,Closed,2019-06-15 12:22:41,NaT,0,,actionfem
4,actionfem-1928-02-15-a,1928,2,15,a,Closed,2019-06-15 12:22:41,NaT,0,,actionfem


The issues table we will use must contain at least the 4 following columns :
- `id`
- `year`
- `access_right`
- `newspaper_id`

### 1.2 Content item table (S3 > dask)
For analysis at the content item level, data has been loaded from a prepared dataset exported from s3. We will show you here how this prepared dataset should look like. 

As it is much larger than the issues dataset, here we load data as a **dask dataframe** (and not a pandas dataframe). <br/> See <a href="https://dask.org/">here</a> for full documentation.

We also use tools from the **impresso_commons** package. <br/>
Full code and documentations can be found <a href="https://github.com/impresso/impresso-pycommons">here</a>.

In [21]:
import dask.dataframe as ddf

In [24]:
# Usefull for reading the dask dataframe in the right format.
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT

In [23]:
# Put the variables corresponding to your own location and file name
PATH = '/scratch/students/justine/'
FILE = 's3-impresso-stats'

ci_df = ddf.read_csv(
    PATH+FILE+"/content-item-stats/*",
    storage_options=IMPRESSO_STORAGEOPT)

##### Snapshot

In [20]:
ci_df.head()

,id,year,newspaper,type,n_tokens,title_length
0,BDC-1839-01-20-a-i0001,1839,BDC,ar,250.0,4.0
1,BDC-1839-01-20-a-i0002,1839,BDC,ar,758.0,NaN
2,BDC-1839-01-20-a-i0003,1839,BDC,ar,14.0,31.0
3,BDC-1839-01-20-a-i0004,1839,BDC,ar,349.0,22.0
4,BDC-1839-01-20-a-i0005,1839,BDC,ar,193.0,NaN


The content item table we will use must contain at least the 5 following columns :
- `id`
- `year`
- `newspaper`
- `type`
- `title_length`

## 2. Dataset statistics

> **This sections aims to show you how to retrieve simple statistics about the datasets, and give you an idea on our dataset scale.**

### 2.1 Issues table (SQL > pandas)

For the pandas dataframe it is rather simple : the functions `shape` and `nunique` enable to get most simple statistics, very quickly. <br/> Note *(for those who are used to use the pandas functions)*: function `describe` is not so relevant here as most columns are not statistical numbers.

##### Shape

In [34]:
newspapers_df.shape

(209, 5)

In [25]:
issues_df.shape

(441861, 11)

##### Content

In [36]:
issues_df.nunique()

id               441861
year                240
month                12
day                  31
edition              18
access_rights         3
created            8862
last_modified        10
is_damaged            2
s3_version            1
newspaper_id         47
dtype: int64

In [39]:
issues_df.year.min(), issues_df.year.max()

(1740, 2018)

In [37]:
issues_df.access_rights.unique()

array(['Closed', 'OpenPublic', 'OpenPrivate'], dtype=object)

##### Conclusion

- The issues dataframe contains 441'861 lines (and 11 columns), each corresponding to one issue. 
- Issues date from 1740 to 2018.
- Issues belong to 47 different newspapers.
- There are 3 types of access rights: 'closed', 'openpublic' and 'openprivate'.

### 2.2 Content item table (S3 > dask)
The content item table is much bigger and statistics are therefor much longer to compute (which is why it is stored as a dask dataframe). <br/>
The following cells may take quite some time to run.

##### Shape

In [40]:
ci_df.shape[0].compute()

47876994

##### Content

In [42]:
ci_df.type.unique().compute()

0         ar
1        img
2         ad
3    section
4    picture
5       page
6         tb
7         ob
8          w
Name: type, dtype: object

##### Conclusion

- The content item table contains 47'876'994 lines (10 times more that the issues one)
- Their type can be one of : ar, img, ad, section, picture, page, tb, ob, w

## 3. Tool functions

> **This section aims to present you useful functions to apply on the tables.**

### 3.1 Expanding

The following functions expand the tables by adding a new column.

#### 3.1.1 Decades
Adding a decade column is useful to get global statistics per decade, and also for plotting. <br/>
Function `decade_from_year_df` does that, and returns a new dataset similar to the one passed as parameter, with a decade column. It can take as parameter both a pandas or a dask dataframe, with a small specification (see below).

**Notes:** 
- The table passed as parameter must have a column `year` to be able to apply the function.
- If the table passed as parameter has already a column named `decade`, the function returns the tables passed as paramater directly.

In [45]:
from impresso_stats.helpers import decade_from_year_df

In [59]:
decade_issues_df = decade_from_year_df(issues_df)

You can see below that the table now has a column `decade`.

In [61]:
decade_issues_df.head()

,id,year,month,day,edition,access_rights,created,last_modified,is_damaged,s3_version,newspaper_id,decade
0,actionfem-1927-10-15-a,1927,10,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem,1920
1,actionfem-1927-11-15-a,1927,11,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem,1920
2,actionfem-1927-12-15-a,1927,12,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem,1920
3,actionfem-1928-01-15-a,1928,1,15,a,Closed,2019-06-15 12:22:41,NaT,0,,actionfem,1920
4,actionfem-1928-02-15-a,1928,2,15,a,Closed,2019-06-15 12:22:41,NaT,0,,actionfem,1920


**With dask dataframe:** When the parameter passed is a dask dataframe, it needs to be specified through the parameter `dask_df`, by setting it to true.

In [74]:
decade_ci_df = decade_from_year_df(ci_df, dask_df=True)

#### 3.1.2 Access rights at content item level
The content item table doesn't contain information about access rights, as the access right type is defined at the issues level. In order to get the access right for each content item, you need to join the two tables. The function `licenses_ci_df` does it directly for you. 

It loads the issues table from SQL, extracts the issues ID in the table passed as parameter, and merges the two.

**Notes:**
- This function is **very sensitive to any change in the table's shape or in the column names** and needs specific entries : 
    - the table passed as parameter must have the following columns (named exactly that way): `'id'`, `'newspaper'`,`'decade'`, `'year'`, `'month'`, `'day'`,  `'type'`, `'n_tokens'`, `'title_length'`, `'access_rights'`. Else you will need to modify the function directly. (In particular, the `decade` column is needed here)
    - its columns `id` must have a specific format (same as the one on the snapshot presented above), from which we can extract the ID of the issue.
    
    - the `issues` table (which is loaded directly in the function) must have the same columns as the one presented earlier in the notebook.
    
- The **number of entries in the merged table is lower that the number of entries in the original content item dataframe**. In fact some newspaper ID are not represented in the issues dataframe (which contains the access rights) and those will therefore be dropped in the merged dataframe as we don't have information on the access right for those (the function does an inner join for this reason). <br/> >> Therefore we recommend using the table output by the function, only when you are doing an analysis related to access rights. For any other analysis we recommend using the original `ci_df` in order to keep as much information as possible.

In [62]:
from impresso_stats.helpers import licenses_ci_df

In [71]:
ci_licences_df = licenses_ci_df(decade_ci_df)

In [72]:
ci_licences_df.shape[0].compute()

33707113

The resulting dataframe has 33'707'113 entries : around 14mio less than the original.

You can see below that the table now has a column `access_rights`.

In [73]:
ci_licences_df.head()

,id,newspaper,decade,year,month,day,type,n_tokens,title_length,access_rights
0,BDC-1839-01-20-a-i0001,BDC,1830,1839,1,20,ar,250.0,4.0,OpenPublic
1,BDC-1839-01-20-a-i0002,BDC,1830,1839,1,20,ar,758.0,NaN,OpenPublic
2,BDC-1839-01-20-a-i0003,BDC,1830,1839,1,20,ar,14.0,31.0,OpenPublic
3,BDC-1839-01-20-a-i0004,BDC,1830,1839,1,20,ar,349.0,22.0,OpenPublic
4,BDC-1839-01-20-a-i0005,BDC,1830,1839,1,20,ar,193.0,NaN,OpenPublic


### 3.2 Filtering

Function `filter_df` takes as parameter a table (dask or pandas) and filtering parameters, and returns a tuple : the filtered table and the IDs of the newspapers which have been kept. It can take the following filters :
- `start_date` and `end_date` : indicate the **years** between which you can to keep the entries.
    - you need to specify **either both either none**: if you specify only one the filter won't be applied.
    - you must have `start_date` <= `end_date` (of course).


- `np_ids` : indicate the specific list of newspapers you want to keep.
    - the list can either be specified by a python list, or by a pandas series.
    
    
- `country` : indicate the country indicator of the newspapers you want to keep (typically 'CH' or 'LU').


- `ppty` and `ppty_value` : indicate the property of the newspapers you want to keep.


**Note:** attributes `country` and `ppty` are defined at the newspaper level (higher level than issues or content items). When specified, the function will load data from the SQL tables `newspapers_metadata` and `meta_properties`. Filtering with these characteristics is done based on the information in those tables. If they are not up-to-date, the output may not be correct.

**For later:** this function is used as a helper function to most visualization functions (those which enable filtering) you will see in tutorials 2 and 3.

In [75]:
from impresso_stats.helpers import filter_df

##### Examples
Examples are shown with `issues_df` for a matter of computation time, but work exactly the same way with a dask dataframe (`ci_df` - in which case a dask dataframe is returned as 1st value of the return tuple).

We provide examples with one specific filter, so that you see how to use each of them. Of course you can **combine** them and specify several filters at the same time.

##### 3.2.1. Filtering by date
Here we keep the issues between 1932 and 1956 (**comprised**).

In [107]:
output_table, np_ids = filter_df(issues_df, start_date=1932, end_date=1956)

In [108]:
output_table.shape

(83930, 11)

In [109]:
output_table.head()

,id,year,month,day,edition,access_rights,created,last_modified,is_damaged,s3_version,newspaper_id
40,actionfem-1932-01-15-a,1932,1,15,a,Closed,2019-06-15 12:22:53,NaT,0,,actionfem
41,actionfem-1932-02-15-a,1932,2,15,a,Closed,2019-06-15 12:22:53,NaT,0,,actionfem
42,actionfem-1932-03-15-a,1932,3,15,a,Closed,2019-06-15 12:22:53,NaT,0,,actionfem
43,actionfem-1932-04-15-a,1932,4,15,a,Closed,2019-06-15 12:22:53,NaT,0,,actionfem
44,actionfem-1932-05-15-a,1932,5,15,a,Closed,2019-06-15 12:22:53,NaT,0,,actionfem


Here are the newspaper IDs which have been kept, i.e. the newspapers which have issues published between 1932 and 1956.

In [110]:
np_ids

array(['actionfem', 'demitock', 'dunioun', 'EXP', 'GDL', 'IMP',
       'indeplux', 'JDG', 'LCE', 'LES', 'LSE', 'luxembourg1935',
       'luxland', 'luxwort', 'NZZ', 'obermosel', 'onsjongen'],
      dtype=object)

##### 3.2.2. Filtering by newspaper IDs
Here we keep only two newspapers : *Gazette de Lausanne* (*GDL*) and *Journal de Genève* (*JDG*).<br/>
(In that case the second return value `np_ids` is not very useful).

In [111]:
output_table, np_ids = filter_df(issues_df, np_ids=['GDL', 'JDG'])

In [112]:
output_table.shape

(100201, 11)

In [113]:
output_table.head()

,id,year,month,day,edition,access_rights,created,last_modified,is_damaged,s3_version,newspaper_id
60257,GDL-1798-02-01-a,1798,2,1,a,OpenPrivate,2019-06-18 07:08:55,2019-06-25 12:48:23,0,,GDL
60258,GDL-1798-02-02-a,1798,2,2,a,OpenPrivate,2019-06-18 07:08:55,2019-06-25 12:48:23,0,,GDL
60259,GDL-1798-02-03-a,1798,2,3,a,OpenPrivate,2019-06-18 07:08:55,2019-06-25 12:48:23,0,,GDL
60260,GDL-1798-02-04-a,1798,2,4,a,OpenPrivate,2019-06-18 07:08:55,2019-06-25 12:48:23,0,,GDL
60261,GDL-1798-02-05-a,1798,2,5,a,OpenPrivate,2019-06-18 07:08:55,2019-06-25 12:48:23,0,,GDL


##### 3.2.2. Filtering by country
Here we keep only the Swiss newspapers - with country value 'CH'. If you want to get the Luxembourgian newspapers, use value 'LU'. 

In [123]:
output_table, np_ids = filter_df(issues_df, country='CH')

In [124]:
output_table.shape

(355466, 11)

In [125]:
output_table.head()

,id,year,month,day,edition,access_rights,created,last_modified,is_damaged,s3_version,newspaper_id
5043,CDV-1843-01-01-a,1843,1,1,a,OpenPublic,2019-06-17 11:53:31,NaT,0,,CDV
5044,CDV-1843-01-04-a,1843,1,4,a,OpenPublic,2019-06-17 11:53:31,NaT,0,,CDV
5045,CDV-1843-01-07-a,1843,1,7,a,OpenPublic,2019-06-17 11:53:31,NaT,0,,CDV
5046,CDV-1843-01-11-a,1843,1,11,a,OpenPublic,2019-06-17 11:53:31,NaT,0,,CDV
5047,CDV-1843-01-14-a,1843,1,14,a,OpenPublic,2019-06-17 11:53:31,NaT,0,,CDV


##### 3.2.2. Filtering by property
TODO